In [1]:
import pandas as pd
import re
from Bio.Seq import Seq

In [2]:
# Gene ids
gene_id_list = ['C5_00660C_A', 'C1_04770C_A', 'C1_02420C_A', 'CR_00850C_A', 'C5_03390C_A',
             'C6_00620W_A', 'C3_05920W_A', 'C3_05220W_A', 'C5_01840C_A', 'C1_08460C_A',
             'C3_04890W_A', 'C3_07860C_A', 'C6_03170C_A', 'C1_08590C_A', 'C3_02220W_A',
             'C1_00800C_A', 'C1_03780C_A', 'C3_06850W_A', 'CR_08780W_A', 'CR_08800W_A',
             'C1_00710C_A', "end"]

gene_names_list = ['Erg11', 'Erg3', 'Fks1', 'Fks2', 'Fur1', 'Fca1', 'Mrr1', 'Cdr1',
                   'Tac1', 'Upc2', 'Cdr2', 'Mrr2', 'Mdr1', 'Erg1', 'Cap1', 'Erg2',
                   'Hmg1', 'Fcr1', 'Its1', 'Its2', 'Tub2', "end"]

In [3]:
test_file = 'test3.table'
#control_file = 'control_albicans.table'

df_test = pd.read_csv(test_file, sep='\t')
#df_control = pd.read_csv(control_file, sep='\t')

df_test = df_test.rename(columns={"NS.2209.003.UDP0161_i7---UDP0161_i5.fRS1_.GT": "sample1.GT", "NS.2209.003.UDP0163_i7---UDP0163_i5.fRS757_.GT": "sample2.GT", "NS.2209.003.UDP0167_i7---UDP0167_i5.A12_.GT" : "sample3.GT"})
#df_control = df_control.rename(columns={"NS.2209.003.UDP0161_i7---UDP0161_i5.fRS1_.GT": "sample1.GT", "NS.2209.003.UDP0163_i7---UDP0163_i5.fRS757_.GT": "sample2.GT", "NS.2209.003.UDP0167_i7---UDP0167_i5.A12_.GT" : "sample3.GT"})

df_test.columns

Index(['CHROM', 'POS', 'REF', 'ALT', 'AC', 'AF', 'AN', 'sample1.GT',
       'sample2.GT', 'sample3.GT'],
      dtype='object')

In [4]:
# Only keep albicans in df_test

df_test_filtered = df_test[df_test['CHROM'] == 'Candidaalbicans']

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,sample3.GT
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,./.
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,./.
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,./.
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,./.
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,./.
...,...,...,...,...,...,...,...,...,...,...
161,Candidaalbicans,51443,TGG,T,2,0.333,6,TGG/TGG,TGG/TGG,T/T
162,Candidaalbicans,51448,G,GAA,2,0.333,6,G/G,G/G,GAA/GAA
163,Candidaalbicans,51460,T,C,2,0.333,6,T/T,T/T,C/C
164,Candidaalbicans,51463,T,C,2,0.333,6,T/T,T/T,C/C


In [5]:
# Exclude variants only called in sample 3. Sample 3 is broken.

def excludeSample(ref, sample1, sample2):
    alt1_1 = re.sub('\/[A-Z]*', '', sample1)
    alt2_1 = re.sub('\/[A-Z]*', '', sample2)
    alt1_2 = re.sub('[A-Z]*\/', '', sample1)
    alt2_2 = re.sub('[A-Z]*\/', '', sample2)
    if (ref == alt1_1) & (ref == alt2_1) & (ref == alt1_2) & (ref == alt2_2):
        return True
    else:
        return False

df_test_filtered['to_drop'] = df_test_filtered.apply(lambda row: excludeSample(row['REF'], row['sample1.GT'], row['sample2.GT']), axis=1)

df_test_filtered = df_test_filtered[df_test_filtered['to_drop'] == False]

df_test_filtered = df_test_filtered.drop(columns=['sample3.GT', 'to_drop'])

df_test_filtered

/tmp/ipykernel_22696/3183125931.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_filtered['to_drop'] = df_test_filtered.apply(lambda row: excludeSample(row['REF'], row['sample1.GT'], row['sample2.GT']), axis=1)


,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A
...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T


In [6]:
# Get gene spans from synthetic genome only albicans ?

synth_gen = '../synth_genome/v4_synthetic_genome.fa'

with open(synth_gen) as genome:
    genome_content = genome.read()

# Removes first line puts a space
#dna = re.sub(">.*", "", genome_content)
genome_string = re.sub("\n", "", genome_content)
albicans_only = re.findall(r">Candidaalbicans\w*", genome_string, flags=re.MULTILINE)[0]

print(len(albicans_only))
print(albicans_only[0:100])

52547
>CandidaalbicansTTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGTTCAGTAGGTAAAACCACCATTGAACTATAATCAGGGTC


In [7]:
# remove header
albicans_only_sequences = re.sub(">Candidaalbicans", "", albicans_only)

print(albicans_only_sequences[0:100])

TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGTTCAGTAGGTAAAACCACCATTGAACTATAATCAGGGTCAGGCACTTTATAACCA


In [8]:
pattern = re.compile("[^N]*N*")
matches = pattern.finditer(albicans_only_sequences)

span_list = []
match_list = []

for i, match in enumerate(matches):
    gene = gene_id_list[i]
    span_list.append(match.span())
    #print(f'span {gene} : ', match.span())
    match_list.append(match.group())
    #print(f'match {gene} : ', match.group())

df_albicans_genes = pd.DataFrame()
df_albicans_genes['Gene ID'] = gene_id_list
df_albicans_genes['Gene name'] = gene_names_list
df_albicans_genes['synth genome positions'] = span_list
df_albicans_genes['match seq'] = match_list

df_albicans_genes

,Gene ID,Gene name,synth genome positions,match seq
0,C5_00660C_A,Erg11,"(0, 1887)",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
1,C1_04770C_A,Erg3,"(1887, 3348)",TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...
2,C1_02420C_A,Fks1,"(3348, 9342)",TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...
3,CR_00850C_A,Fks2,"(9342, 14565)",CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...
4,C5_03390C_A,Fur1,"(14565, 15522)",TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...
5,C6_00620W_A,Fca1,"(15522, 16345)",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...
6,C3_05920W_A,Mrr1,"(16345, 19972)",ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...
7,C3_05220W_A,Cdr1,"(19972, 24778)",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...
8,C5_01840C_A,Tac1,"(24778, 28024)",TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...
9,C1_08460C_A,Upc2,"(28024, 30463)",CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...


In [9]:
gene_seq_list = []

for match in match_list:
    gene_seq = re.sub("N", "", match)
    gene_seq_list.append(gene_seq)

df_albicans_genes['dna seq'] = gene_seq_list

df_albicans_genes

,Gene ID,Gene name,synth genome positions,match seq,dna seq
0,C5_00660C_A,Erg11,"(0, 1887)",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
1,C1_04770C_A,Erg3,"(1887, 3348)",TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...
2,C1_02420C_A,Fks1,"(3348, 9342)",TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...
3,CR_00850C_A,Fks2,"(9342, 14565)",CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...
4,C5_03390C_A,Fur1,"(14565, 15522)",TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...
5,C6_00620W_A,Fca1,"(15522, 16345)",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...
6,C3_05920W_A,Mrr1,"(16345, 19972)",ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...
7,C3_05220W_A,Cdr1,"(19972, 24778)",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...
8,C5_01840C_A,Tac1,"(24778, 28024)",TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...
9,C1_08460C_A,Upc2,"(28024, 30463)",CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...


In [10]:
### get exons

def getExons(geneID):
    
    gff_file = '../synth_genome/gff_files/C_albicans_SC5314_version_A22-s07-m01-r195.gff'
    
    with open(gff_file) as gff:
        gff_content = gff.read()
    
    # gene range
    y = re.findall(f"^.*\t.*\tgene\t.*?{geneID}", gff_content, flags=re.MULTILINE)
    coordinates = re.findall("[^\\t]+", y[0])
    #print(coordinates)
    gene_strand = coordinates[6]
    #print(gene_strand)
    gene_begin = int(coordinates[3]) -1 # Minus 1 to get to base 0 
    gene_end = int(coordinates[4]) # Here we do not substract because we want to include the last nucleotide.
    gene_range = gene_end - gene_begin
    #print(gene_range)
    
    # exons ranges
    w = re.findall(f"^.*\t.*\texon\t.*?{geneID}", gff_content, flags=re.MULTILINE)
    exon_relCoord_list = []
    for index, j in enumerate(w):
        exon_coord = re.findall("[^\\t]+", w[index])
        exon_begin = int(exon_coord[3]) - 1 - gene_begin
        exon_end = int(exon_coord[4]) - gene_begin
        exon_relCoord_list.append((exon_begin, exon_end))
    return exon_relCoord_list

#testing
#getExons(df_albicans_genes.iloc[5]['Gene ID'])
#print(len(df_albicans_genes.iloc[5]['dna seq']))

df_albicans_genes['exon coords'] = df_albicans_genes.apply(lambda row: getExons(row['Gene ID']), axis=1)

df_albicans_genes


,Gene ID,Gene name,synth genome positions,match seq,dna seq,exon coords
0,C5_00660C_A,Erg11,"(0, 1887)",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
1,C1_04770C_A,Erg3,"(1887, 3348)",TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,"[(0, 1161)]"
2,C1_02420C_A,Fks1,"(3348, 9342)",TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,"[(0, 5694)]"
3,CR_00850C_A,Fks2,"(9342, 14565)",CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,"[(0, 4923)]"
4,C5_03390C_A,Fur1,"(14565, 15522)",TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,"[(0, 657)]"
5,C6_00620W_A,Fca1,"(15522, 16345)",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]"
6,C3_05920W_A,Mrr1,"(16345, 19972)",ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,"[(0, 3327)]"
7,C3_05220W_A,Cdr1,"(19972, 24778)",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]"
8,C5_01840C_A,Tac1,"(24778, 28024)",TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,"[(0, 2946)]"
9,C1_08460C_A,Upc2,"(28024, 30463)",CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,"[(0, 2139)]"


In [11]:
### get Strand

def getStrand(geneID):
    
    gff_file = '../synth_genome/gff_files/C_albicans_SC5314_version_A22-s07-m01-r195.gff'
    
    with open(gff_file) as gff:
        gff_content = gff.read()
    
    # gene range
    y = re.findall(f"^.*\t.*\tgene\t.*?{geneID}", gff_content, flags=re.MULTILINE)
    coordinates = re.findall("[^\\t]+", y[0])
    #print(coordinates)
    gene_strand = coordinates[6]
    return gene_strand

df_albicans_genes['strand'] = df_albicans_genes.apply(lambda row: getStrand(row['Gene ID']), axis=1)

df_albicans_genes


,Gene ID,Gene name,synth genome positions,match seq,dna seq,exon coords,strand
0,C5_00660C_A,Erg11,"(0, 1887)",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",-
1,C1_04770C_A,Erg3,"(1887, 3348)",TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,"[(0, 1161)]",-
2,C1_02420C_A,Fks1,"(3348, 9342)",TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,"[(0, 5694)]",-
3,CR_00850C_A,Fks2,"(9342, 14565)",CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,"[(0, 4923)]",-
4,C5_03390C_A,Fur1,"(14565, 15522)",TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,"[(0, 657)]",-
5,C6_00620W_A,Fca1,"(15522, 16345)",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",+
6,C3_05920W_A,Mrr1,"(16345, 19972)",ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,"[(0, 3327)]",+
7,C3_05220W_A,Cdr1,"(19972, 24778)",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",+
8,C5_01840C_A,Tac1,"(24778, 28024)",TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,"[(0, 2946)]",-
9,C1_08460C_A,Upc2,"(28024, 30463)",CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,"[(0, 2139)]",-


In [12]:
def getExonSeq(Exon_coord, dna_seq):
    Exon_seqs_list = []
    for i in range(len(Exon_coord)):
        Exon_seqs_list.append(dna_seq[Exon_coord[0][0]:Exon_coord[0][1]])
    #print(Exon_seqs_list)
    s = ""
    return s.join(Exon_seqs_list)

#getExonSeq(df_albicans_genes.iloc[5]['exon coords'], df_albicans_genes.iloc[5]['dna seq'])

df_albicans_genes['exon seq'] = df_albicans_genes.apply(lambda row: getExonSeq(row['exon coords'], row['dna seq']), axis=1)

df_albicans_genes

,Gene ID,Gene name,synth genome positions,match seq,dna seq,exon coords,strand,exon seq
0,C5_00660C_A,Erg11,"(0, 1887)",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",-,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
1,C1_04770C_A,Erg3,"(1887, 3348)",TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...,"[(0, 1161)]",-,TCATTGTTCAACATATTCTCTATCGTCAACTTTACCTTCAACTTCT...
2,C1_02420C_A,Fks1,"(3348, 9342)",TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...,"[(0, 5694)]",-,TTAAACTCTGAATGGATTTGTAGAATAAGGGGTGGTAGTTCCAGTA...
3,CR_00850C_A,Fks2,"(9342, 14565)",CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...,"[(0, 4923)]",-,CTAGTAATGCTTAATTTCTAATTTTTTCTTTAATCCTTTCAGATTT...
4,C5_03390C_A,Fur1,"(14565, 15522)",TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...,"[(0, 657)]",-,TTAAATACAGTAATATCTATCACCGAAATCACCTAGACCTGGAACA...
5,C6_00620W_A,Fca1,"(15522, 16345)",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",+,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...
6,C3_05920W_A,Mrr1,"(16345, 19972)",ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...,"[(0, 3327)]",+,ATGTCAATTGCCACCACCCCTATAGAAACACCTAAAAGTCCTAAAA...
7,C3_05220W_A,Cdr1,"(19972, 24778)",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",+,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...
8,C5_01840C_A,Tac1,"(24778, 28024)",TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...,"[(0, 2946)]",-,TTAAATCCCCAAATTATTGTCAAAGAAAAAATTGGGTAAATTATTA...
9,C1_08460C_A,Upc2,"(28024, 30463)",CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...,"[(0, 2139)]",-,CTATTTCATATTCATAAACCCATTATCACCTTGCATTAAGTCTAAA...


In [13]:
def getGeneName(mutation_pos):
    for i in range(0,21):
        if ((int(mutation_pos) >= int(df_albicans_genes.iloc[i]['synth genome positions'][0])) & (int(mutation_pos) <= int(df_albicans_genes.iloc[i]['synth genome positions'][1]))):
            #print(df_albicans_genes.iloc[i]['synth genome positions'][0], df_albicans_genes.iloc[i]['synth genome positions'][1])
            return df_albicans_genes.iloc[i]['Gene name']

df_test_filtered['gene or protein name'] = df_test_filtered.apply(lambda row: getGeneName(row['POS']), axis=1)
df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11
...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1


In [14]:
def AddStrand(mutation_pos):
    for i in range(0,21):
        if ((int(mutation_pos) >= int(df_albicans_genes.iloc[i]['synth genome positions'][0])) & (int(mutation_pos) <= int(df_albicans_genes.iloc[i]['synth genome positions'][1]))):
            #print(df_albicans_genes.iloc[i]['synth genome positions'][0], df_albicans_genes.iloc[i]['synth genome positions'][1])
            return df_albicans_genes.iloc[i]['strand']

df_test_filtered['strand'] = df_test_filtered.apply(lambda row: AddStrand(row['POS']), axis=1)
df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-
...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-


In [15]:
def getSynthLandmark(mutation_pos):
    for i in range(0,21):
        if ((int(mutation_pos) >= int(df_albicans_genes.iloc[i]['synth genome positions'][0])) & (int(mutation_pos) <= int(df_albicans_genes.iloc[i]['synth genome positions'][1]))):
            #print(df_albicans_genes.iloc[i]['synth genome positions'][0], df_albicans_genes.iloc[i]['synth genome positions'][1])
            return df_albicans_genes.iloc[i]['synth genome positions'][0]

In [16]:
df_test_filtered['synth_landmark'] = df_test_filtered.apply(lambda row: getSynthLandmark(row['POS']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236


In [17]:
def getMutationPosition(pos, landmark):
    return (pos - landmark - 1) # base 0 strings

In [18]:
df_test_filtered['Mutation_pos_in_dna_seq'] = df_test_filtered.apply(lambda row: getMutationPosition(row['POS'], row['synth_landmark']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766


In [19]:
def getDNAseq(landmark):
    for i in range(0,21):
        if landmark == df_albicans_genes.iloc[i]['synth genome positions'][0]:
            return df_albicans_genes.iloc[i]['dna seq']

In [20]:
df_test_filtered['dna_seq'] = df_test_filtered.apply(lambda row: getDNAseq(row['synth_landmark']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...


In [21]:
def getExonseqForMergedDF(landmark):
    for i in range(0,21):
        if landmark == df_albicans_genes.iloc[i]['synth genome positions'][0]:
            return df_albicans_genes.iloc[i]['exon seq']

In [22]:
df_test_filtered['exon seq'] = df_test_filtered.apply(lambda row: getExonseqForMergedDF(row['synth_landmark']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...


In [23]:
def mutate_dna_sequence(dna_sequence, position, alt_aa):
    # Create a Biopython Seq object
    seq = Seq(dna_sequence)
    # Mutate the sequence at the specified position
    #mutated_seq = seq[:position-1] + alt_aa + seq[position:]
    mutated_seq = seq[:position] + alt_aa + seq[position+1:]
    return str(mutated_seq)

In [24]:
numbers = [0,1,2,3,4,5,6,7,8,9]

for i, n in enumerate(numbers):
    print(i, n)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


In [25]:
df_test_filtered['mutated_dna_seq'] = df_test_filtered.apply(lambda row: mutate_dna_sequence(row['dna_seq'], row['Mutation_pos_in_dna_seq'], row['ALT']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...


In [26]:
# Proof of mutation
print(df_test_filtered.iloc[0]['dna_seq'][117])

print(df_test_filtered.iloc[0]['mutated_dna_seq'][117])

A
G


In [27]:
### Get mutated exon sequence

# Add exon coords to merged df...
def getExonCoordForMergedDF(landmark):
    for i in range(0,21):
        if landmark == df_albicans_genes.iloc[i]['synth genome positions'][0]:
            return df_albicans_genes.iloc[i]['exon coords']
        
df_test_filtered['exon_coords'] = df_test_filtered.apply(lambda row: getExonCoordForMergedDF(row['synth_landmark']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq,exon_coords
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]"
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]"
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]"
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]"


In [28]:
# Get mutated exon seq

#df_test_filtered['mutated exon seq'] = df_test_filtered.apply(lambda row: getExonseqForMergedDF(row['dna seq'], row['Mutation pos in dna seq'], row['ALT']), axis=1)
#df_test_filtered

df_test_filtered['mutated_exon_seq'] = df_test_filtered.apply(lambda row: getExonSeq(row['exon_coords'], row['mutated_dna_seq']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq,exon_coords,mutated_exon_seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...


In [29]:
def translate(exon_sequence, strand):
    # Create a Biopython Seq object
    seq = Seq(exon_sequence)
    if strand == '-':
        qes = seq.reverse_complement() ############### REVERSE to update car pas tous les genes doivent l'etre
        aa_sequence = qes.translate(table=12)
    else:
        aa_sequence = seq.translate(table=12)
        
    return str(aa_sequence)

In [30]:
df_test_filtered['AA_seq'] = df_test_filtered.apply(lambda row: translate(row['exon seq'], row['strand']), axis=1)
df_test_filtered['mutated_AA_seq'] = df_test_filtered.apply(lambda row: translate(row['mutated_exon_seq'], row['strand']), axis=1)
df_test_filtered

/home/labolandry/anaconda3/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/home/labolandry/anaconda3/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,strand,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq,exon_coords,mutated_exon_seq,AA_seq,mutated_AA_seq
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,-,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,-,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,-,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,-,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,GYC*NCH*WH*LFFVP*CYTTDQYIIRGSICLQLSMAIFIFIKKR*...
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,-,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,-,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,-,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,-,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...


In [31]:
len(df_test_filtered.iloc[0]['exon seq'])

1587

In [32]:
df_test_filtered.iloc[0]['AA_seq']

'MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKDRAPLVFYWIPWFGSAASYGQQPYEFFESCRQKYGDVFSFMLLGKIMTVYLGPKGHEFVFNAKLSDVSAEDAYKHLTTPVFGKGVIYDCPNSRLMEQKKFAKFALTTDSFKRYVPKIREEILNYFVTDESFKLKEKTHGVANVMKTQPEITIFTASRSLFGDEMRRIFDRSFAQLYSDLDKGFTPINFVFPNLPLPHYWRRDAAQKKISATYMKEIKSRRERGDIDPNRDLIDSLLIHSTYKDGVKMTDQEIANLLIGILMGGQHTSASTSAWFLLHLGEKPHLQDVIYQEVVELLKEKGGDLNDLTYEDLQKLPSVNNTIKETLRMHMPLHSIFRKVTNPLRIPETNYIVPKGHYVLVSPGYAHTSERYFDNPEDFDPTRWDTAAAKANSVSFNSSDEVDYGFGKVSKGVSSPYLPFGGGRHRCIGEQFAYVQLGTILTTFVYNLRWTIDGYKVPDPDYSSMVVLPTEPAEIIWEKRETCMF*'

In [33]:
def getDeltaAA(WT, MUT):
    for i, aa in enumerate(list(MUT)):
        initial = list(WT)[i]
        if aa != initial:
            return f'{initial}{i+1}{aa}'
    
    return 0

In [34]:
df_test_filtered['mutation'] = df_test_filtered.apply(lambda row: getDeltaAA(row['AA_seq'], row['mutated_AA_seq']), axis=1)

df_test_filtered

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,...,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq,exon_coords,mutated_exon_seq,AA_seq,mutated_AA_seq,mutation
1,Candidaalbicans,118,A,G,2,0.500,4,A/G,A/G,Erg11,...,0,117,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,0
2,Candidaalbicans,148,T,C,2,0.500,4,T/C,T/C,Erg11,...,0,147,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,0
3,Candidaalbicans,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,...,0,181,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,0
4,Candidaalbicans,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,...,0,185,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,GYC*NCH*WH*LFFVP*CYTTDQYIIRGSICLQLSMAIFIFIKKR*...,M1G
5,Candidaalbicans,478,G,A,2,0.500,4,G/A,G/A,Erg11,...,0,477,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,"[(0, 1587)]",TTAAAACATACAAGTTTCTCTTTTTTCCCAAATGATTTCTGCTGGT...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,MAIVETVIDGINYFLSLSVTQQISILLGVPFVYNLVWQYLYSLRKD...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Candidaalbicans,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,...,44236,2307,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,0
156,Candidaalbicans,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,...,44236,2478,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,0
157,Candidaalbicans,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,...,44236,2715,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,0
158,Candidaalbicans,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,...,44236,2766,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,"[(0, 3222)]",CTAAGATTTAATACATGTTACTGAACCTTCTTCAAGACGTTTCAAA...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,MLSFLTEVTGAIAQTSARRPIQFMVVPALLASIAYLSIIDDYIPEH...,0


In [35]:
df_test_filtered[df_test_filtered['strand'] == '+']

,CHROM,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,...,synth_landmark,Mutation_pos_in_dna_seq,dna_seq,exon seq,mutated_dna_seq,exon_coords,mutated_exon_seq,AA_seq,mutated_AA_seq,mutation
31,Candidaalbicans,15694,T,C,2,0.500,4,T/C,T/C,Fca1,...,15522,171,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,0
32,Candidaalbicans,15699,A,G,2,0.500,4,A/G,A/G,Fca1,...,15522,176,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,0
33,Candidaalbicans,15802,A,G,2,0.500,4,A/G,A/G,Fca1,...,15522,279,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,0
34,Candidaalbicans,15811,A,G,2,0.500,4,A/G,A/G,Fca1,...,15522,288,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,"[(0, 55), (125, 523)]",ATGACGTTTGACGACAAAAAAGGTTTACAAGTTGCTCTTGATCAAG...,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,MTFDDKKGLQVALDQAKKNDV*RQKRFTSCS*SSQE,0
35,Candidaalbicans,22480,T,C,2,0.500,4,T/C,T/C,Cdr1,...,19972,2507,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,0
36,Candidaalbicans,22585,A,G,2,0.500,4,A/G,A/G,Cdr1,...,19972,2612,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,0
37,Candidaalbicans,22633,A,T,2,0.500,4,A/T,A/T,Cdr1,...,19972,2660,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,0
38,Candidaalbicans,22678,C,T,2,0.333,6,C/T,C/T,Cdr1,...,19972,2705,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,"[(0, 4506)]",ATGTCAGATTCTAAGATGTCGTCGCAAGATGAATCTAAATTAGAAA...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,MSDSKMSSQDESKLEKAISQDSSSENHSINEYHGFDAHTSENIQNL...,0
71,Candidaalbicans,33541,A,G,2,0.500,4,A/G,A/G,Cdr2,...,30463,3077,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,"[(0, 4500)]",ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,MSTANTSLSQQLDEKPWVDASDNSSVQEYQGFDATASHNIQDLARK...,MSTANTSLSQQLDEKPWVDASDNSSVQEYQGFDATASHNIQDLARK...,0
72,Candidaalbicans,33544,T,A,2,0.500,4,T/A,T/A,Cdr2,...,30463,3080,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,"[(0, 4500)]",ATGAGTACTGCAAACACGTCTTTGTCGCAACAGCTAGACGAAAAGC...,MSTANTSLSQQLDEKPWVDASDNSSVQEYQGFDATASHNIQDLARK...,MSTANTSLSQQLDEKPWVDASDNSSVQEYQGFDATASHNIQDLARK...,0


In [36]:
def getMutationPosOnly(mutation):
    string = re.sub("[^\d]", "", mutation)
    return string

In [37]:
def getAltAAOnly(mutation):
    return re.sub(".+\d+", "", mutation)

In [38]:
df_cleaned = df_test_filtered.drop(columns=['CHROM', 'dna_seq', 'exon seq', 'mutated_dna_seq', 'exon_coords', 'mutated_exon_seq', 'synth_landmark', 'Mutation_pos_in_dna_seq', 'AA_seq', 'mutated_AA_seq', 'strand'])

df_cleaned['species'] = 'Candida albicans'

#df_cleaned['MutPos'] = df_cleaned.apply(lambda row: getMutationPosOnly(str(row['mutation'])), axis=1)
#df_cleaned['Alt_AA'] = df_cleaned.apply(lambda row: getAltAAOnly(str(row['mutation'])), axis=1)

df_cleaned

,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,mutation,species
1,118,A,G,2,0.500,4,A/G,A/G,Erg11,0,Candida albicans
2,148,T,C,2,0.500,4,T/C,T/C,Erg11,0,Candida albicans
3,182,CTA,C,1,0.250,4,CTA/CTA,CTA/C,Erg11,0,Candida albicans
4,186,G,GGT,1,0.250,4,G/G,G/GGT,Erg11,M1G,Candida albicans
5,478,G,A,2,0.500,4,G/A,G/A,Erg11,0,Candida albicans
...,...,...,...,...,...,...,...,...,...,...,...
155,46544,T,C,2,0.500,4,T/C,T/C,Hmg1,0,Candida albicans
156,46715,C,T,2,0.500,4,C/T,C/T,Hmg1,0,Candida albicans
157,46952,G,A,2,0.500,4,G/A,G/A,Hmg1,0,Candida albicans
158,47003,C,T,2,0.500,4,C/T,C/T,Hmg1,0,Candida albicans


In [39]:
df_database = pd.read_csv("FungAMR.csv")

df_database = df_database[['species', 'gene or protein name', 'mutation', 'drug', 'degree of evidence']]

#df_database['MutPos'] = df_database.apply(lambda row: getMutationPosOnly(str(row['mutation'])), axis=1)
#df_database['Alt_AA'] = df_database.apply(lambda row: getAltAAOnly(str(row['mutation'])), axis=1)

df_database

/tmp/ipykernel_22696/1272542406.py:1: DtypeWarning: Columns (7,8,13,17,24,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_database = pd.read_csv("FungAMR.csv")


,species,gene or protein name,mutation,drug,degree of evidence
0,Aspergillus niger,Cyp51A,R228Q,Posaconazole,8
1,Tapesia acuformis,Cyp51,Y486H,Prochloraz,7
2,Tapesia acuformis,Cyp51,Y486H,Flusilazole,7
3,Tapesia acuformis,Cyp51,Y486H,Triadimenol,7
4,Aspergillus fumigatus,Cyp51A,I266N,Itraconazole,8
...,...,...,...,...,...
54661,Saccharomyces cerevisiae,Fks1,"D1353Y,R1357V",Micafungin,-3
54662,Saccharomyces cerevisiae,Fks1,"D1353Y,R1357W",Micafungin,3
54663,Saccharomyces cerevisiae,Fks1,"D1353Y,R1356S",Micafungin,-3
54664,Saccharomyces cerevisiae,Fks1,"D1353Y,R1356V",Micafungin,-3


In [40]:
df_merged = df_cleaned.merge(df_database, how='inner', on=['species', 'mutation' , 'gene or protein name'], suffixes=('_pipeline', '_database'))

df_merged = df_merged.drop_duplicates()

df_merged

,POS,REF,ALT,AC,AF,AN,sample1.GT,sample2.GT,gene or protein name,mutation,species,drug,degree of evidence
0,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Fluconazole,-8
1,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Itraconazole,-8
2,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Ketoconazole,8
4,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Fluconazole,-2
5,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Fluconazole,8
8,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Clotrimazole,-3
9,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Fluconazole,-3
10,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Isavuconazole,-3
11,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Itraconazole,-3
12,1205,T,G,2,0.500,4,T/G,T/G,Erg11,K128T,Candida albicans,Posaconazole,-3


In [41]:
df_merged_cleaned = df_merged[['species', 'POS','REF', 'ALT', 'sample1.GT', 'sample2.GT', 'gene or protein name', 'mutation', 'drug']]

df_merged_cleaned = df_merged_cleaned.rename(columns={"species": "Species", "POS": "Position in gene", "REF":"nucleotide WT", "ALT":"nucleotide mutation", "sample1.GT": "Genotype sample 1", "sample2.GT":"Genotype sample 2"})

#df_merged_cleaned['drug'] = 'Azoles'

df_merged_cleaned = df_merged_cleaned.drop_duplicates().reset_index(drop=True)

df_merged_cleaned

,Species,Position in gene,nucleotide WT,nucleotide mutation,Genotype sample 1,Genotype sample 2,gene or protein name,mutation,drug
0,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Fluconazole
1,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Itraconazole
2,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Ketoconazole
3,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Clotrimazole
4,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Isavuconazole
5,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Posaconazole
6,Candida albicans,1205,T,G,T/G,T/G,Erg11,K128T,Voriconazole
7,Candida albicans,1240,A,T,A/T,A/T,Erg11,D116E,Fluconazole
8,Candida albicans,1240,A,T,A/T,A/T,Erg11,D116E,Itraconazole
9,Candida albicans,1240,A,T,A/T,A/T,Erg11,D116E,Voriconazole
